# Demo usage niche_vlaanderen

Niche Vlaanderen can be used interactively in Python. For interactive use, we recommend using the jupyter notebook. This allows one to save code and results in the same file.

This file itself is a notebook, and can be accessed at: [demo.ipynb](https://github.com/inbo/niche_vlaanderen/blob/master/docs/demo.ipynb)

## Getting started

The first step is importing the `niche_vlaanderen` module. For convenience, we will be importing as `nv`.

In [ ]:
import niche_vlaanderen as nv

Next we create a `Niche` object. This object will hold all the data and results for an analysis. We will call the instance simple, as we will be using a [simple model](https://inbo.github.io/niche_vlaanderen/vegetatie.html#eenvoudig-niche-model), using only MHW, MLW and soil for the predictions.

In [ ]:
simple = nv.Niche()

After initialization, we can add input layers to this object, using the `set_input` method.

In [ ]:
simple.set_input("mhw","../testcase/zwarte_beek/input/mhw.asc")
simple.set_input("mlw","../testcase/zwarte_beek/input/mhw.asc")
simple.set_input("soil_code","../testcase/zwarte_beek/input/soil_code.asc")

These three input files are the minimal inputs required for running a simple NICHE model. This means we can already run our model.

In [ ]:
simple.run(full_model=False)

After a model is run, we can inspect the results using the `table` method. Note that the values are given in m². 

In [ ]:
import pandas as pd
td=list()
for i in simple._vegetation:
    vi = pd.Series(simple._vegetation[i].flatten())
    rec =  vi.value_counts() * simple._context.cell_area
    #print(rec)
    for a in rec.index:
        td.append((i,a,rec[a]))
simple.table.head()

The returned object is a [pandas dataframe](https://pandas.pydata.org/pandas-docs/stable/10min.html) which makes it easy to manipulate (eg crosstab) or save it to a file.

In [ ]:
simple.table.to_csv("demo.csv")

By using the pandas [pivot_table](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot_table.html#pandas.DataFrame.pivot_table) method, we can create a summarized table. Note that only the first 5 rows are shown because we use the [head](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html) function

In [ ]:
result = simple.table.pivot_table(index="vegetation", 
                            values="area", 
                            columns="presence",
                            fill_value=0).head()

result.head()

It is also possible to show actual grids using the `plot` method.

In [ ]:
simple.plot(11)
import matplotlib.pyplot as plt
plt.show()

In [ ]:
simple.name="test"
simple.plot(11)

plt.show()

## Running a full Niche model

A full Niche model requires more inputs that only mhw, mlw and soil_code. The full list can be found in the documentation at xxx. It is also possible to look at the allowed_inputs set

In [ ]:
nv.niche.minimal_input()

In [ ]:
full = nv.Niche()
path = "../testcase/zwarte_beek/input/"
full.set_input("mhw", path + "mhw.asc")
full.set_input("mlw", path + "mlw.asc")
full.set_input("soil_code",path + "soil_code.asc")
full.set_input("nitrogen_animal", 0)
full.set_input("inundation_acidity", path + "inundation.asc")
full.set_input("inundation_nutrient", path+ "inundation.asc")
full.set_input("nitrogen_fertilizer",0)
full.set_input("minerality", path + "minerality.asc")
full.set_input("management", path + "management.asc")
full.set_input("nitrogen_atmospheric", 0)
full.set_input("msw", path + "msw.asc")
full.set_input("rainwater",0)
full.set_input("seepage", path + "seepage.asc")
full.run()

In [ ]:
full.table.head()

## Comparing Niche classes

Niche models can be compared using a NicheDelta model. 

In [ ]:
delta = nv.NicheDelta(simple, full)
ax = delta.plot(7)
plt.show()

It is also possible to show the areas in a table (dataframe).

In [ ]:
delta.table.head()

In [ ]:
delta.write("_output")

## Creating deviation maps

To create deviation maps, it is necessary to run a model with the deviation option.

In [ ]:
dev = nv.Niche()
dev.set_input("mhw","../testcase/zwarte_beek/input/mhw.asc")
dev.set_input("mlw","../testcase/zwarte_beek/input/mhw.asc")
dev.set_input("soil_code","../testcase/zwarte_beek/input/soil_code.asc")
dev.run(deviation=True, full_model=False)

In [ ]:
dev.plot("mhw_14")
plt.show()

## Creating statistics per shape object

Niche also contains a helper function that allows one to calculate the possible vegetation by using a vector dataset, such as a .geojson or .shp file.

The vegetation is returned as a dict of pandas dataframes (one for every shape).

In [ ]:
df = full.zonal_stats("../testcase/zwarte_beek/input/study_area_l72.geojson")
df.head()